#DV360 Deal Finder
Compares open vs. deal CPM, CPC, and CPA so that clients can decide which sites, inventory, and deals work best.


#License

Copyright 2020 Google LLC,

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

  https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.



#Disclaimer
This is not an officially supported Google product. It is a reference implementation. There is absolutely NO WARRANTY provided for using this code. The code is Apache Licensed and CAN BE fully modified, white labeled, and disassembled by your team.

This code generated (see starthinker/scripts for possible source):
  - **Command**: "python starthinker_ui/manage.py colab"
  - **Command**: "python starthinker/tools/colab.py [JSON RECIPE]"



#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Set Configuration

This code is required to initialize the project. Fill in required fields and press play.

1. If the recipe uses a Google Cloud Project:
  - Set the configuration **project** value to the project identifier from [these instructions](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md).

1. If the recipe has **auth** set to **user**:
  - If you have user credentials:
    - Set the configuration **user** value to your user credentials JSON.
  - If you DO NOT have user credentials:
    - Set the configuration **client** value to [downloaded client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md).

1. If the recipe has **auth** set to **service**:
  - Set the configuration **service** value to [downloaded service credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_service.md).



In [ ]:
from starthinker.util.configuration import Configuration


CONFIG = Configuration(
  project="",
  client={},
  service={},
  user="/content/user.json",
  verbose=True
)



#3. Enter DV360 Deal Finder Recipe Parameters
 1. Wait for **BigQuery->->->Deal_Finder_Dashboard** to be created.
 1. Join the [StarThinker Assets Group](https://groups.google.com/d/forum/starthinker-assets) to access the following assets
 1. Copy [Deal Finder Sample Data](https://datastudio.google.com/open/1QrWNTurvQT6nx20vnzdDveSzSmRjqHxQ).
 1. Click Edit Connection, and change to **BigQuery->StarThinker Data->->Deal_Finder_Dashboard**.
 1. Copy [Deal Finder Sample Report](https://datastudio.google.com/open/1fjRI5AIKTYTA4fWs-pYkJbIMgCumlMyO).
 1. When prompted choose the new data source you just created.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'recipe_slug':'',  # Place where tables will be written in BigQuery.
  'recipe_timezone':'America/Los_Angeles',  # Timezone for report dates.
  'recipe_name':'',  # Name of report in DV360, should be unique.
  'auth_write':'service',  # Credentials used for writing data.
  'auth_read':'user',  # Credentials used for reading data.
  'partners':[],  # DV360 partner id.
  'advertisers':[],  # Comma delimited list of DV360 advertiser ids.
}

print("Parameters Set To: %s" % FIELDS)


#4. Execute DV360 Deal Finder
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

TASKS = [
  {
    'dataset':{
      'description':'Create a dataset for bigquery tables.',
      'hour':[
        4
      ],
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'dbm':{
      'description':'Create a DV360 report.',
      'hour':[
        3
      ],
      'auth':{'field':{'name':'auth_read','kind':'authentication','order':1,'default':'user','description':'Credentials used for reading data.'}},
      'report':{
        'filters':{
          'FILTER_PARTNER':{
            'values':{'field':{'name':'partners','kind':'integer_list','order':5,'default':[],'description':'DV360 partner id.'}}
          },
          'FILTER_ADVERTISER':{
            'values':{'field':{'name':'advertisers','kind':'integer_list','order':6,'default':[],'description':'Comma delimited list of DV360 advertiser ids.'}}
          }
        },
        'body':{
          'timezoneCode':{'field':{'name':'recipe_timezone','kind':'timezone','description':'Timezone for report dates.','default':'America/Los_Angeles'}},
          'metadata':{
            'title':{'field':{'name':'recipe_name','kind':'string','prefix':'Deal Finder For ','description':'Name of report in DV360, should be unique.'}},
            'dataRange':'LAST_30_DAYS',
            'format':'CSV'
          },
          'params':{
            'type':'TYPE_CROSS_PARTNER',
            'groupBys':[
              'FILTER_PARTNER_NAME',
              'FILTER_PARTNER',
              'FILTER_ADVERTISER_NAME',
              'FILTER_ADVERTISER',
              'FILTER_APP_URL',
              'FILTER_SITE_ID',
              'FILTER_INVENTORY_SOURCE_NAME',
              'FILTER_INVENTORY_SOURCE',
              'FILTER_INVENTORY_SOURCE_TYPE',
              'FILTER_ADVERTISER_CURRENCY',
              'FILTER_CREATIVE_WIDTH',
              'FILTER_CREATIVE_HEIGHT',
              'FILTER_CREATIVE_TYPE'
            ],
            'metrics':[
              'METRIC_IMPRESSIONS',
              'METRIC_CLICKS',
              'METRIC_TOTAL_CONVERSIONS',
              'METRIC_TOTAL_MEDIA_COST_ADVERTISER',
              'METRIC_REVENUE_ADVERTISER',
              'METRIC_ACTIVE_VIEW_MEASURABLE_IMPRESSIONS',
              'METRIC_ACTIVE_VIEW_VIEWABLE_IMPRESSIONS'
            ]
          }
        }
      }
    }
  },
  {
    'dbm':{
      'description':'Copy a DV360 report to BigQuery.',
      'hour':[
        4
      ],
      'auth':{'field':{'name':'auth_read','kind':'authentication','order':1,'default':'user','description':'Credentials used for reading data.'}},
      'report':{
        'name':{'field':{'name':'recipe_name','kind':'string','prefix':'Deal Finder For ','description':'Name of report in DV360, should be unique.'}},
        'timeout':10
      },
      'out':{
        'bigquery':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be written in BigQuery.'}},
          'table':'Deal_Finder_DV360_Report',
          'header':True,
          'schema':[
            {
              'name':'Partner',
              'type':'STRING'
            },
            {
              'name':'Partner_ID',
              'type':'INTEGER'
            },
            {
              'name':'Advertiser',
              'type':'STRING'
            },
            {
              'name':'Advertiser_ID',
              'type':'INTEGER'
            },
            {
              'name':'Site',
              'type':'STRING'
            },
            {
              'name':'Site_ID',
              'type':'INTEGER'
            },
            {
              'name':'Inventory',
              'type':'STRING',
              'mode':'NULLABLE'
            },
            {
              'name':'Inventory_ID',
              'type':'INTEGER',
              'mode':'NULLABLE'
            },
            {
              'name':'Inventory_Type',
              'type':'STRING'
            },
            {
              'name':'Advertiser_Currency',
              'type':'STRING'
            },
            {
              'name':'Creative_Width',
              'type':'STRING',
              'mode':'NULLABLE'
            },
            {
              'name':'Creative_Height',
              'type':'STRING',
              'mode':'NULLABLE'
            },
            {
              'name':'Creative_Type',
              'type':'STRING'
            },
            {
              'name':'Impressions',
              'type':'INTEGER'
            },
            {
              'name':'Clicks',
              'type':'INTEGER'
            },
            {
              'name':'Conversions',
              'type':'FLOAT'
            },
            {
              'name':'Cost',
              'type':'FLOAT'
            },
            {
              'name':'Revenue',
              'type':'FLOAT'
            },
            {
              'name':'AV_Impressions_Measurable',
              'type':'INTEGER'
            },
            {
              'name':'AV_Impressions_Viewable',
              'type':'INTEGER'
            }
          ]
        }
      }
    }
  },
  {
    'bigquery':{
      'description':'The logic query for Deal Finder, transforms report into view used by datastudio.',
      'hour':[
        4
      ],
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'from':{
        'query':"SELECT Partner, Partner_ID, Advertiser, Advertiser_ID, Site, Site_ID, Inventory, Inventory_Type, Creative_Type, Creative_Size, Always_On, Deal_Impressions, Open_Impressions, Rank_Impressions, Deal_Clicks, Open_Clicks, Rank_Clicks, Deal_Conversions, Open_Conversions, Rank_Conversions, Deal_Impressions_Viewable, Open_Impressions_Viewable, Rank_Impressions_Viewable, Deal_Impressions_Measurable, Open_Impressions_Measurable, Rank_Impressions_Measurable, Deal_Cost, Open_Cost, Rank_Cost, FROM ( SELECT FIRST(Partner) AS Partner, FIRST(Partner_ID) AS Partner_ID, FIRST(Advertiser) AS Advertiser, Advertiser_ID, First(Site) AS Site, Site_ID, Inventory, Inventory_Type, Creative_Type, Creative_Width + ' x ' + Creative_Height AS Creative_Size, IF (LEFT(Inventory, 5) == 'AO - ', True, False) AS Always_On, SUM(Deal_Impressions) AS Deal_Impressions, SUM(Open_Impressions) AS Open_Impressions, SUM(Open_Impressions) + SUM(Deal_Impressions) AS Total_Impressions, ROW_NUMBER() OVER (PARTITION BY Advertiser_ID ORDER BY Total_Impressions DESC) AS Rank_Impressions, SUM(Deal_Clicks) AS Deal_Clicks, SUM(Open_Clicks) AS Open_Clicks, SUM(Open_Clicks) + SUM(Deal_Clicks) AS Total_Clicks, ROW_NUMBER() OVER (PARTITION BY Advertiser_ID ORDER BY Total_Clicks DESC) AS Rank_Clicks, SUM(Deal_Conversions) AS Deal_Conversions, SUM(Open_Conversions) AS Open_Conversions, SUM(Open_Conversions) + SUM(Deal_Conversions) AS Total_Conversions, ROW_NUMBER() OVER (PARTITION BY Advertiser_ID ORDER BY Total_Conversions DESC) AS Rank_Conversions, SUM(Deal_Cost) AS Deal_Cost, SUM(Open_Cost) AS Open_Cost, SUM(Open_Cost) + SUM(Deal_Cost) AS Total_Cost, RANK() OVER (PARTITION BY Advertiser_ID ORDER BY Total_Cost DESC) AS Rank_Cost, SUM(Deal_Impressions_Viewable) AS Deal_Impressions_Viewable, SUM(Open_Impressions_Viewable) AS Open_Impressions_Viewable, SUM(Open_Impressions_Viewable) + SUM(Deal_Impressions_Viewable) AS Total_Impressions_Viewable, ROW_NUMBER() OVER (PARTITION BY Advertiser_ID ORDER BY Total_Impressions_Viewable DESC) AS Rank_Impressions_Viewable, SUM(Deal_Impressions_Measurable) AS Deal_Impressions_Measurable, SUM(Open_Impressions_Measurable) AS Open_Impressions_Measurable, SUM(Open_Impressions_Measurable) + SUM(Deal_Impressions_Measurable) AS Total_Impressions_Measurable, ROW_NUMBER() OVER (PARTITION BY Advertiser_ID ORDER BY Total_Impressions_Measurable DESC) AS Rank_Impressions_Measurable, FROM ( SELECT Partner, Partner_ID, Advertiser, Advertiser_ID, Site, Site_ID, Inventory, Inventory_Type, Creative_Type, Creative_Width, Creative_Height, IF(Inventory_ID IS NULL, Impressions, 0) AS Open_Impressions, IF(Inventory_ID IS NULL, 0, Impressions) AS Deal_Impressions, IF(Inventory_ID IS NULL, Clicks, 0) AS Open_Clicks, IF(Inventory_ID IS NULL, 0, Clicks) AS Deal_Clicks, IF(Inventory_ID IS NULL, Conversions, 0) AS Open_Conversions, IF(Inventory_ID IS NULL, 0, Conversions) AS Deal_Conversions, IF(Inventory_ID IS NULL, Cost, 0) AS Open_Cost, IF(Inventory_ID IS NULL, 0, Cost) AS Deal_Cost, IF(Inventory_ID IS NULL, AV_Impressions_Viewable, 0) AS Open_Impressions_Viewable, IF(Inventory_ID IS NULL, 0, AV_Impressions_Viewable) AS Deal_Impressions_Viewable, IF(Inventory_ID IS NULL, AV_Impressions_Measurable, 0) AS Open_Impressions_Measurable, IF(Inventory_ID IS NULL, 0, AV_Impressions_Measurable) AS Deal_Impressions_Measurable, FROM [[PARAMETER].Deal_Finder_DV360_Report] OMIT RECORD IF Site == 'Low volume inventory') GROUP By Advertiser_ID, Site_ID, Inventory, Inventory_Type, Creative_Type, Creative_Size, Always_On) WHERE Rank_Impressions < 100 OR Rank_Clicks < 100 OR Rank_Conversions < 100 OR Rank_Cost < 100;",
        'parameters':[
          {'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be written in BigQuery.'}}
        ]
      },
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be written in BigQuery.'}},
        'view':'Deal_Finder_Dashboard'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(CONFIG, TASKS, force=True)
